# Example 2

**This is adapted from https://github.com/svenreiche/Genesis-1.3-Version4/tree/master/examples/Example2-Dumps**

The internal data of genesis is represented by wavefronts for the fields and the 6D particle distribution for electrons. They contain more information than the output of the main output file.

Genesis supports to export the information to external files, also called dumps.

This example will show the three methods to utilize dumps.

In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np

from genesis.version4 import Genesis4

%config InlineBackend.figure_format = 'retina'  # Nicer plots

## Markers
Markers in the lattice file can trigger the output of either field or particle distribution. To enable outputting the 
lattice file from `Example1.lat` is modified with these lines (replacing the previous definition of the FODO cell of example 1):

```
M: MARKER = {dumpfield = 1};
FODO: LINE={UND,D1,QF,D2,UND,D1,QD,D2,M};
```

Markers are zero length element and do not add to the total length of the lattice. In this case the Marker is added to the end of the FODO cell.
With 6 cells in the full lattice there will be 6 field dumps. They are prefixed with the `rootname` (as in the `setup` namelist) and have the extension 
```.fld.h5```. To differentiate the different outputs the name has also the integration step number in between the `rootname` and the extension. For example, integration step 184 would have the filename ```Example2.184.fld.h5```.

## Dumps Outside of Tracking

The namelist `write` in the main input file can write the field or particle distribution at any point outside of the tracking.
In this example, the input file has the `write` namelist:
```
&write
field = dump
&end
```
This will write the field distribution with the name ```dump.fld.h5```. The extension is included automatically, so you do not specify it in the namelist.

Since Genesis has executed the tracking before, the field distribution represents the state at the end of the tracked beamline. In our case this is the end of the undulator.

## Periodic Output While Tracking

The last method for dumping data is to define in the `track` namelist the number of integration steps after which a dump is issued.
For this example, we modify it to output after every second integration step:

```
&track
beam_dump_step = 2
&end
```
In total about 600 files will be generated with this method, with the filename suffixes including the integration step number.

Note that generating dumps at each step can cause to a large number of files. In particular for time-dependent runs,
the size per dump can be large (on the order of gigabytes) and generating hundreds of them may fill up your disk faster than you expect.


## Running the example

Running ```Example2.in``` causes the additional output when ever a dump file is generated. Also for a better illustraiton of the output, the
energy spread of the beam has been reduced to 
```
delgam=0.100000
```
for a better display of the FEL process.

In [ ]:
G = Genesis4("data/example2-dumps/Example2.in")

In [ ]:
G.verbose = True
output = G.run()

In [ ]:
G.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"], figsize=(8, 8))

In [ ]:
print("Loaded fields:", output.load_fields())
print("Total loaded particles files=", len(output.load_particles()))

# Wavefront

Wavefront distribution after the first 4 FODO cells. Note that the output seems noisy, which comes from the limited number of macro particles in the simulation and  the reduced diffraction in the Angstrom wavelength regime.

In [ ]:
def get_slice(field, slc: int) -> np.ndarray:
    return field.dfl[:, :, slc]


def get_wavefront(field, slice=0):
    # inten = np.reshape(fre * fre + fim * fim, (ng, ng))
    inten = np.abs(get_slice(field, slice) ** 2)
    return inten, field.param.gridsize * (field.param.gridpoints - 1) * 0.5 * 1e3


# plot wavefront
istep = 184
_, axs = plt.subplots(2, 2, figsize=(8, 8))
color = "yellow"
for i1 in range(2):
    for i2 in range(2):
        i = (i2 * 2 + i1 + 1) * istep
        inten, dg = get_wavefront(output.field3d[i], slice=0)
        axs[i2, i1].imshow(inten, extent=(-dg, dg, -dg, dg))
        txt = r"$z$ = %3.1f m" % (9.5 * (i2 * 2 + i1 + 1))
        axs[i2, i1].text(-0.15, 0.15, txt, color=color)

axs[1, 0].set_xlabel(r"$x$ (mm)")
axs[1, 1].set_xlabel(r"$x$ (mm)")
axs[0, 0].set_ylabel(r"$y$ (mm)")
axs[1, 0].set_ylabel(r"$y$ (mm)")
plt.show()

## Longitudinal Phasespace Distribution

This is snapshot of the longitudinal phase space of the electron beam towards the end of the undulator beamline. The x-axis coordinate is the ponderomotive phase, which is given by the longidutinal position multiplied with the wavenumber of the resonant wavelength. In this case, it corresponds to a slice thickness of 1 Angstrom.

In [ ]:
def get_phase_space(particles, lambda0):
    theta = np.mod(particles.z / lambda0, 1) * 2 * np.pi
    return theta, particles.energy


# get range for phase space plots
emin = np.min(output.beam.emin)
emax = np.max(output.beam.emax)
xmin = np.min(output.beam.xmin)
xmax = np.max(output.beam.xmax)

# plot final phase space
theta, energy = get_phase_space(
    output.particles[700], lambda0=G.input.main.setup.lambda0
)
plt.scatter(theta, energy / 1e9, s=0.2)
plt.xlabel(r"$\theta$ (rad)")
plt.ylabel(r"$E$ (GeV)")
plt.ylim(xmin, xmax)
plt.ylim(emin / 1e9, emax / 1e9)
plt.show()

# Animate phase space

## Animation 1

With the dump of the electron phase space after every second integration step, the python script generates two animation. One for the longitudinal phase space, similar to the plot above.  The other is the distribution in the horizontal plane.

Here the formation of the micro bunching can be seen with its initial modulation in energy to roughly a quarter rotation in phase space. A significant part of electrons are not modulated. These corresponds for electrons with large betatron amplitude, which cannot stay in resonance with the radiation field.

In [ ]:
from IPython.display import Video

fig = plt.figure()
ax = plt.axes(xlim=(0, 2 * np.pi), ylim=(emin / 1e9, emax / 1e9))
ax.set_xlabel(r"$\theta$ (rad)")
ax.set_ylabel(r"$E$ (GeV)")
scat = ax.scatter([], [], s=0.2)


def init():
    scat.set_offsets(np.empty((0, 2)))
    return (scat,)


def animate(i):
    theta, energy = get_phase_space(
        output.particles[2 * i], lambda0=G.input.main.setup.lambda0
    )
    scat.set_offsets(np.hstack((theta[:, np.newaxis], energy[:, np.newaxis] / 1e9)))
    return (scat,)


# We could also get individual frames by doing the following:
# animate(1)

anim = animation.FuncAnimation(
    fig,
    animate,
    init_func=init,
    blit=False,
    interval=20,
    frames=500,
)
anim.save("Animation1.mp4")

# Hide our plot and then show the final animation:
plt.close()
Video("Animation1.mp4")

## Animation 2

The breathing in the transverse size comes from the focusing of the FODO lattice. Note that electrons with large amplitude have the tendency to fall backwards quickly. They cannot stay in resonance and thus do not get modulated in energy.

The small jerks in the animation corresponds to the phase shifter, since in the drift there is a phase velocity mismatch between field and beam. To correct that a phase shifter adjust the phase. In Genesis this is done automatically via `autophase`.

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, 2 * np.pi), ylim=(xmin * 1e6, xmax * 1e6))
ax.set_xlabel(r"$\theta$ (rad)")
ax.set_ylabel(r"$x$ ($\mu$m)")
scat = ax.scatter([], [])


def animate2(i):
    particles = output.particles[2 * i]
    theta, _ = get_phase_space(particles, lambda0=G.input.main.setup.lambda0)
    scat.set_offsets(
        np.hstack((theta[:, np.newaxis], particles.x[:, np.newaxis] * 1e6))
    )
    return (scat,)


# We could also get individual frames by doing the following:
# animate2(1)

anim = animation.FuncAnimation(
    fig,
    animate2,
    init_func=init,
    blit=False,
    interval=20,
    frames=500,
)
anim.save("Animation2.mp4")

# Hide our plot and then show the final animation:
plt.close()
Video("Animation2.mp4")